In [ ]:
%load_ext autoreload
%autoreload 2
from fenics import *
import numpy as np
from .BiotStatic import solve_static_biot
import sympy as sym

set_log_level(16)

u_degree = 2
p_degree = 1
resolutions = [8, 16, 32, 64, 128]

expr_degree = 4
material_parameter = dict()
material_parameter["K"] = 1e-8
material_parameter["lmbda"] = 1e12
material_parameter["alpha"] = 1

boundary_all = CompiledSubDomain("on_boundary")
boundary_right = CompiledSubDomain("on_boundary && near(x[0], 1)")
boundary_left = CompiledSubDomain("on_boundary && near(x[0], 0)")
boundary_top = CompiledSubDomain("on_boundary && near(x[1], 1)")
boundary_bottom = CompiledSubDomain("on_boundary && near(x[1], 0)")


def compute_order(error, h):
    h = np.array(h)
    err_ratio = np.array(error[:-1]) / np.array(error[1:])
    return np.log(err_ratio)/np.log(h[:-1] / h[1:])

In [ ]:
from sympy import symbols, pi
from sympy import derive_by_array, tensorcontraction

lmbda, alpha, t, K = sym.symbols('lmbda, alpha, t, K')
K = material_parameter["K"]

x,y,z = sym.symbols("x, y, z")

ux = (sym.sin(2*pi*x) + sym.sin(2*pi*y))
uy = (sym.cos(2*pi*x) + sym.sin(2*pi*y))

u = sym.Matrix([ux, uy, 0])
p = (sym.cos(2*pi*x) * sym.sin(2*pi*y))

def row_wise_div(tensor):
    return sym.Matrix(tensorcontraction(derive_by_array( tensor, [x,y,z]), (0,2)))
    
def row_wise_grad(u):
    return u.jacobian([x,y,z])

def eps(u):
    return 0.5*(row_wise_grad(u) + sym.transpose(row_wise_grad(u)))

def div(u):
    return sym.trace(u.jacobian([x,y,z]))

def grad(p):
    return sym.Matrix([sym.diff(p,x), sym.diff(p,y), sym.diff(p,z)])
 
f = - row_wise_div(eps(u) + lmbda*div(u)*sym.eye(3) - alpha*p*sym.eye(3))
g = - ((alpha**2)/lmbda)*p - alpha*div(u) + div(K*grad(p))

f.simplify()
g.simplify()

p_N_right = - sym.diff( p, x).subs(x, 1)
p_N_left = sym.diff(p, x).subs(x, 0)
p_N_bottom = sym.diff(p, y).subs(y, 0)
p_N_top = - sym.diff(p, y).subs(y, 1)


fx, fy = f[0], f[1]
ux, uy = u[0], u[1]

In [ ]:
xcpp, ycpp = sym.symbols('x[0], x[1]')

variables_scalar = [g, p, p_N_right, p_N_left, p_N_bottom, p_N_top]
variables_scalar_cpp  = [sym.printing.ccode(v.subs(x, xcpp).subs(y, ycpp)) for v in variables_scalar]
variables_scalar_expr  = [Expression(v, **material_parameter, t=0, degree=expr_degree)
                          for v in variables_scalar_cpp]
g, p_e, p_N_right, p_N_left, p_N_bottom, p_N_top = variables_scalar_expr


variables_vector = [f, u]
variables_vector_cpp  = [(sym.printing.ccode(v[0].subs(x, xcpp).subs(y, ycpp)),
                          sym.printing.ccode(v[1].subs(x, xcpp).subs(y, ycpp)))
                          for v in variables_vector]
variables_vector_expr  = [Expression(v, **material_parameter, t=0, degree=expr_degree)
                          for v in variables_vector_cpp]
f, u_e = variables_vector_expr


In [ ]:
def biot_convergence(resolutions, boundaries,
                     boundary_conditions_p,
                     boundary_conditions_u,
                     solve_iterative,
                     solver_params=None):
    L2_errors_u = []
    L2_errors_p = []
    h = []
    for N in resolutions:
        mesh = UnitSquareMesh(N, N, diagonal="left")
        h.append(mesh.hmax())

        boundary_marker = MeshFunction("size_t", mesh, mesh.topology().dim()-1, value=0)
        
        for b, marker_id in boundaries.items():
            b.mark(boundary_marker, marker_id)
            
        u, pT, p = solve_static_biot(mesh, f, g,
                                       material_parameter,
                                       boundary_marker, boundary_conditions_p,
                                       boundary_marker, boundary_conditions_u,
                                       u_degree=u_degree, p_degree=p_degree,
                                       solver_params=solver_params,
                                       solve_iterative=solve_iterative)
                
            
        J_p = errornorm(p_e, p, "L2",)
        J_u = errornorm(u_e, u, "L2",)

        L2_errors_u.append(J_u)
        L2_errors_p.append(J_p)

    return L2_errors_u, L2_errors_p, h, solution    


In [ ]:
# Test convergence of direct solver
boundaries = {boundary_all: 1,}

boundary_conditions_p = {1:{"Dirichlet": p_e},}
boundary_conditions_u = {1:{"Dirichlet":u_e},}

res = biot_convergence(resolutions, boundaries, 
                       boundary_conditions_p,
                       boundary_conditions_u,
                       solve_iterative=False)
L2_errors_u, L2_errors_p, h, solution = res  

print(f"L2 EOC for u: {compute_order(L2_errors_u, h)}")
print(f"L2 EOC for p: {compute_order(L2_errors_p, h)}")
print(f"L2 error for u: {L2_errors_u}")
print(f"L2 error for p: {L2_errors_p}")


In [ ]:
# Test convergence of iterative solver
solver_params={"relativeconv":False, "tolerance":1e-18, "maxiter":200}

res = biot_convergence(resolutions, boundaries, 
                       boundary_conditions_p,
                       boundary_conditions_u,
                       solve_iterative=True,
                       solver_params=solver_params)
L2_errors_u, L2_errors_p, h, solution = res  

print(f"L2 EOC for u: {compute_order(L2_errors_u, h)}")
print(f"L2 EOC for p: {compute_order(L2_errors_p, h)}")
print(f"L2 error for u: {L2_errors_u}")
print(f"L2 error for p: {L2_errors_p}")
